# Versión mejorada

In [9]:
import numpy as np
import random
from joblib import Parallel, delayed

# Lectura de datos desde archivo
data = np.loadtxt('qap.datos/tai256c.dat', skiprows=1)

# Separación en matrices de flujos y distancias
flow = np.int32(data[:256])
distances = np.int32(data[256:])

# Configuración de la semilla para reproducibilidad
seed = 2024
random.seed(seed)
np.random.seed(seed)

# Generación de población inicial de 5000 individuos
population_size = 5000
population = np.array([np.random.permutation(np.arange(256)) for _ in range(population_size)])

# Función de fitness eficiente usando operaciones matriciales
def fitness_pop(population, flow, distances):
    return np.sum(flow[np.newaxis, :, :] * distances[population[:,:,np.newaxis], population[:, np.newaxis, :]], axis=(1,2))

# Selección por torneo con elitismo y control de nichos
def selection_tournament_with_niches(population, fitness_vals, k=3, elite_ratio=0.1, niche_size=5):
    elite_size = int(len(population) * elite_ratio)
    elite_indices = np.argsort(fitness_vals)[:elite_size]
    elites = population[elite_indices]

    niches = [random.sample(range(len(population)), niche_size) for _ in range(len(population) - elite_size)]
    selected_indices = [min(niche, key=lambda x: fitness_vals[x]) for niche in niches]
    selected = population[selected_indices]

    return np.vstack((elites, selected))

# Cruce basado en ciclos (Cycle Crossover, CX)
def crossover_cycle(parent1, parent2):
    n = len(parent1)
    child = -np.ones(n, dtype=int)
    start = 0
    indices = set()

    while start not in indices:
        indices.add(start)
        start = np.where(parent2 == parent1[start])[0][0]

    for idx in indices:
        child[idx] = parent1[idx]

    for idx in range(n):
        if child[idx] == -1:
            child[idx] = parent2[idx]

    return child

def crossover_population(population, crossover_prob=0.9):
    new_population = []
    for _ in range(len(population) // 2):
        parent1, parent2 = population[random.sample(range(len(population)), 2)]
        if random.random() < crossover_prob:
            child1 = crossover_cycle(parent1, parent2)
            child2 = crossover_cycle(parent2, parent1)
        else:
            child1, child2 = parent1.copy(), parent2.copy()
        new_population.extend([child1, child2])
    return np.array(new_population)

# Mutación avanzada: Scramble Mutation + Adaptive Mutation
def mutate_population_advanced(population, mutation_prob, stagnation_counter, max_stagnation):
    adaptive_prob = mutation_prob + (0.1 * stagnation_counter / max_stagnation)
    adaptive_prob = min(adaptive_prob, 0.5)

    for individual in population:
        if random.random() < adaptive_prob:
            if random.random() < 0.5:
                # Scramble Mutation
                i, j = sorted(random.sample(range(len(individual)), 2))
                np.random.shuffle(individual[i:j+1])
            else:
                # Swap Mutation
                i, j = random.sample(range(len(individual)), 2)
                individual[i], individual[j] = individual[j], individual[i]
    return population

# Búsqueda Local Mejorada: 3-opt
def local_search_3opt(permutation, flow, distances, max_iter=50):
    def calculate_cost(permutation):
        indexed_distances = distances[permutation][:, permutation]
        return np.sum(flow * indexed_distances)

    n = len(permutation)
    best_cost = calculate_cost(permutation)
    for _ in range(max_iter):
        i, j, k = sorted(random.sample(range(n), 3))
        new_perm = permutation.copy()
        new_perm[i:j+1] = list(reversed(new_perm[i:j+1]))
        new_perm[j+1:k+1] = list(reversed(new_perm[j+1:k+1]))
        new_cost = calculate_cost(new_perm)
        if new_cost < best_cost:
            permutation = new_perm
            best_cost = new_cost
    return permutation

def parallel_local_search(population, flow, distances, elite_ratio=0.1, max_iter=50):
    elite_size = int(len(population) * elite_ratio)
    elite_indices = np.argsort(fitness_pop(population, flow, distances))[:elite_size]
    elite_population = population[elite_indices]

    improved_elites = Parallel(n_jobs=-1)(
        delayed(local_search_3opt)(individual, flow, distances, max_iter)
        for individual in elite_population
    )

    population[elite_indices] = improved_elites
    return population

# Enfriamiento simulado
def simulated_annealing(permutation, flow, distances, initial_temp=1000, cooling_rate=0.99, max_iter=50):
    def calculate_cost(permutation):
        indexed_distances = distances[permutation][:, permutation]
        return np.sum(flow * indexed_distances)

    current_temp = initial_temp
    best_perm = permutation.copy()
    best_cost = calculate_cost(permutation)

    for _ in range(max_iter):
        i, j = sorted(random.sample(range(len(permutation)), 2))
        new_perm = permutation.copy()
        new_perm[i:j+1] = list(reversed(new_perm[i:j+1]))
        new_cost = calculate_cost(new_perm)
        if new_cost < best_cost or random.random() < np.exp(-(new_cost - best_cost) / current_temp):
            permutation = new_perm
            best_cost = new_cost
            best_perm = permutation
        current_temp *= cooling_rate
    return best_perm

# Ciclo evolutivo
generations = 400
crossover_prob = 0.9
mutation_prob = 0.3
stagnation_limit = 100

best_solution = None
best_fitness = float('inf')
no_improvement_counter = 0

fitness_vals = fitness_pop(population, flow, distances)

for generation in range(generations):
    if no_improvement_counter >= stagnation_limit // 2:
        new_individuals = np.array([np.random.permutation(np.arange(256)) for _ in range(len(population) // 10)])
        population = np.vstack((population, new_individuals))
        fitness_vals = fitness_pop(population, flow, distances)  # Recalcular fitness

    population = parallel_local_search(population, flow, distances)
    fitness_vals = fitness_pop(population, flow, distances)  # Asegurarse de que esté sincronizado
    selected_population = selection_tournament_with_niches(population, fitness_vals)
    offspring = crossover_population(selected_population, crossover_prob)
    population = mutate_population_advanced(offspring, mutation_prob, no_improvement_counter, stagnation_limit)

    fitness_vals = fitness_pop(population, flow, distances)  # Calcular nuevamente el fitness
    gen_best_fitness = fitness_vals.min()

    if gen_best_fitness < best_fitness:
        best_fitness = gen_best_fitness
        best_solution = population[np.argmin(fitness_vals)]
        no_improvement_counter = 0
    else:
        no_improvement_counter += 1

    print(f"Generación {generation + 1}: Mejor Fitness = {best_fitness}")
    if no_improvement_counter >= stagnation_limit:
        print("Estancamiento detectado, deteniendo el proceso.")
        break


print("Mejor solución encontrada:", best_solution)
print("Costo asociado:", best_fitness)


Generación 1: Mejor Fitness = 49640646
Generación 2: Mejor Fitness = 48936998
Generación 3: Mejor Fitness = 48644572
Generación 4: Mejor Fitness = 48572670
Generación 5: Mejor Fitness = 48572670
Generación 6: Mejor Fitness = 48400640
Generación 7: Mejor Fitness = 48231486
Generación 8: Mejor Fitness = 48079376
Generación 9: Mejor Fitness = 48079376
Generación 10: Mejor Fitness = 48079376
Generación 11: Mejor Fitness = 48021320
Generación 12: Mejor Fitness = 48011438
Generación 13: Mejor Fitness = 47979872
Generación 14: Mejor Fitness = 47630182
Generación 15: Mejor Fitness = 47612504
Generación 16: Mejor Fitness = 47612504
Generación 17: Mejor Fitness = 47494014
Generación 18: Mejor Fitness = 47255906
Generación 19: Mejor Fitness = 47255906
Generación 20: Mejor Fitness = 47205734
Generación 21: Mejor Fitness = 47111630
Generación 22: Mejor Fitness = 47086418
Generación 23: Mejor Fitness = 46968058
Generación 24: Mejor Fitness = 46864536
Generación 25: Mejor Fitness = 46800122
Generació